In [1]:
from ase import Atoms
from mace.calculators import mace_mp
from ase.build import surface
from ase.constraints import FixAtoms, UnitCellFilter
from ase.optimize import QuasiNewton
from ase.visualize import view
from ase.io import read, write
from ase.optimize import LBFGS
import numpy as np
from cu2o_bulk import cu2o_bulk, cu2o111, CuD_FCC111, py111

b = cu2o_bulk()
b.calc = mace_mp(model="medium", dispersion=False, default_dtype="float64", device='cpu')
E_bulk = b.get_potential_energy()

slab = py111(3, 10)
slab.calc = mace_mp(model="medium", dispersion=False, default_dtype="float64", device='cpu')
view(slab)

E_slab = slab.get_potential_energy()
E_cleav = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])
print(f'{n_layers=} {E_cleav=}')

bottom_Cu_z = np.min(slab[slab.symbols=='Cu'].positions[:,2])
mask1=slab.positions[:, 2] < bottom_Cu_z + 1.0
slab.set_constraint(FixAtoms(mask=mask1))

qn = LBFGS(slab, trajectory='111slab.traj')
qn.run(fmax=0.01)
E_slab = slab.get_potential_energy()
t = read('111slab.traj@:')
atoms = t[-1]
view(atoms)
 
E_surf = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])
print(f'{n_layers=} {E_surf=}')


"\nE_slab = slab.get_potential_energy()\nE_cleav = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])\nprint(f'{n_layers=} {E_cleav=}')\n\nbottom_Cu_z = np.min(slab[slab.symbols=='Cu'].positions[:,2])\nmask1=slab.positions[:, 2] < bottom_Cu_z + 1.0\nslab.set_constraint(FixAtoms(mask=mask1))\n\nqn = LBFGS(slab, trajectory='111slab.traj')\nqn.run(fmax=0.01)\nE_slab = slab.get_potential_energy()\nt = read('111slab.traj@:')\natoms = t[-1]\nview(atoms)\n \nE_surf = (E_slab - E_bulk * n_layers) / 2 / np.linalg.det(slab.cell[:2, :2])\nprint(f'{n_layers=} {E_surf=}')\n"

In [34]:
O_pos = np.mean(slab.positions[slab.positions[:,2] > 15, :], axis=0) + [2.35,1.3, 3.75]
Cu_pos = np.mean(slab.positions[slab.positions[:,2] > 15, :], axis=0) + [1.0,0.5, 2.0]
O = Atoms(symbols='O', positions = [O_pos])
Cu = Atoms(symbols='Cu', positions = [Cu_pos])

Cu2 = Cu.copy()
Cu2.translate([2.8,0,0])
Cu3 = Cu.copy()
Cu3.translate([1.3,2.5,0])

slab_py = slab + O + Cu + Cu2 + Cu3
view(slab_py)

<Popen: returncode: None args: ['/home/lana/ve/bin/python3', '-m', 'ase', 'g...>